In [6]:
%pip install crewai
%pip install crewai-tools


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
import os
from crewai import LLM

WATSONX_URL = "https://us-south.ml.cloud.ibm.com"
WATSONX_API_KEY = ""
WATSONX_PROJECT_ID = ""
WATSONX_MODEL_ID = "watsonx/meta-llama/llama-3-8b-instruct"
SERPER_API_KEY = ""

os.environ["WATSONX_URL"] = WATSONX_URL  # (required) Base URL of your WatsonX instance
os.environ["WATSONX_APIKEY"] = WATSONX_API_KEY  # IBM cloud API key (required)
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID  # Project ID of your WatsonX instance (required)
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

llm = LLM(
    model=WATSONX_MODEL_ID,
    base_url=WATSONX_URL,
    project_id=WATSONX_PROJECT_ID,
    max_tokens=2000,
    temperature=0.7
)

In [32]:
from crewai import Agent, Task, Crew, Process

data_collector = Agent(
    role='Data Collector',
    goal='Collect accurate and up-to-date financial data',
    backstory='You are an expert in gathering financial data from various sources.',
    tools=[scrape_tool, search_tool],
    verbose=True,
    allow_delegation=True,
    llm=llm
)

financial_analyst = Agent(
    role='Financial Analyst',
    goal='Analyze financial data and provide insights',
    backstory='You are a seasoned financial analyst with years of experience in interpreting market trends.',
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

report_writer = Agent(
    role='Report Writer',
    goal='Compile findings into a comprehensive report',
    backstory='You are skilled at creating clear and concise financial reports.',
    llm=llm
)

In [33]:
# Define the tasks
data_collector_task = Task(
    description='Collect stock data for {company_name} from their company website {company_website} and yahoo finance site {yahoo_finance} for the past month',
    expected_output="""
        A list with all stock data for {company_name} from their company website {company_website} 
        and yahoo finance site {yahoo_finance} for the past month collected
    """,
    agent=data_collector
)


In [34]:

financial_analyst_task = Task(
    description='Analyze the collected data and identify trends',
    expected_output="""
        Analyze the collected data and identify trends
    """,
    agent=financial_analyst
)
report_writer_task = Task(
    description='Write a comprehensive report on the financial analysis',
    expected_output="""
       Write a comprehensive report on the financial analysis'
    """,
    agent=report_writer
)

# Assemble the crew
financial_crew = Crew(
    agents=[data_collector, financial_analyst, report_writer],
    tasks=[data_collector_task, financial_analyst_task, report_writer_task],
    process=Process.sequential
)

# Execute the analysis
inputs = {
    'company_name': 'Tesla Inc.',
    'company_website': 'https://www.tesla.com/',
    'yahoo_finance': 'https://finance.yahoo.com/quote/TSLA'
}

2024-12-03 10:43:32,835 - 8213613568 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [35]:
result = financial_crew.kickoff(inputs)
print(result)

# Agent: Data Collector
## Task: Collect stock data for Tesla Inc. from their company website https://www.tesla.com/ and yahoo finance site https://finance.yahoo.com/quote/TSLA for the past month


# Agent: Data Collector
## Thought: Thought: I need to start by gathering the stock data from both the company website and Yahoo Finance site for the past month. I'll begin by searching the company website for any relevant information.
## Using tool: Search the internet
## Tool Input: 
"[{\"search_query\": \"tesla stock data\"}, {\"website_url\": \"https://finance.yahoo.com/quote/TSLA\"}]"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Data Collector
## Thought: Let's start again.
Thought: I need to start by gathering the stock data from both the company website and Yahoo Finance site for the past month. I'll begin by searching the company website for any relevant information.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"